In [ ]:
#import necessary libraries
import numpy as np # lin alg
import matplotlib.pyplot as plt # plot
import pandas as pd # data input
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

data = pd.read_csv('IRIS.csv')
data.describe()

In [ ]:
sns.pairplot(data, hue= 'species', height= 3)
plt.show()

#### This is the code for the neural network

In [ ]:
class NeuralNetwork:
    def __init__(self, n_input, n_hidden, n_outputs, learning_rate):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_outputs = n_outputs
        self.learning_rate = learning_rate
        
        # Initialize weights and biases with random values
        self.weights = [np.random.randn(self.n_hidden, self.n_input),
                        np.random.randn(self.n_outputs, self.n_hidden)]
        
        self.biases = [np.random.randn(self.n_hidden),
                       np.random.randn(self.n_outputs)]

    def sigmoid(self, x):
        x = np.clip(x, -500, 500)  # Clip input values to avoid overflow
        return 1 / (1 + np.exp(-x))

    def dsigmoid(self, x):
        return x * (1 - x)

    def make_prediction(self, input_vector):
        hidden_layer = self.sigmoid(np.dot(self.weights[0], input_vector) + self.biases[0])
        output_layer = self.sigmoid(np.dot(self.weights[1], hidden_layer) + self.biases[1])
        return output_layer

    def gradient_descent(self, input_vector, target):
        # Forward pass
        hidden_layer = self.sigmoid(np.dot(self.weights[0], input_vector) + self.biases[0])
        output_layer = self.sigmoid(np.dot(self.weights[1], hidden_layer) + self.biases[1])
        
        # Calculate error
        error = output_layer - target
        
        # Backpropagation
        derror_doutput = error * self.dsigmoid(output_layer)
        doutput_dhidden = np.dot(self.weights[1].T, derror_doutput) * self.dsigmoid(hidden_layer)

        # Calculate gradients
        derror_dweights1 = np.outer(derror_doutput, hidden_layer)
        derror_dbias1 = derror_doutput
        derror_dweights0 = np.outer(doutput_dhidden, input_vector)
        derror_dbias0 = doutput_dhidden

        return [derror_dweights0, derror_dweights1], [derror_dbias0, derror_dbias1]

    def update(self, gradients, biases):
        self.weights[0] -= self.learning_rate * gradients[0]
        self.weights[1] -= self.learning_rate * gradients[1]
        self.biases[0] -= self.learning_rate * biases[0]
        self.biases[1] -= self.learning_rate * biases[1]

    def train(self, input_vectors, targets, iterations):
        all_errors = []
        for current_iteration in range(iterations):
            # Pick a random data point
            random_index = np.random.randint(len(input_vectors))
            input_vector = input_vectors[random_index]
            target = targets[random_index]

            # Compute gradients using backpropagation
            gradients, biases = self.gradient_descent(input_vector, target)

            # Update weights and biases
            self.update(gradients, biases)

            # Calculate cumulative error for monitoring
            if current_iteration % 100 == 0:
                cumulative_error = 0
                for i in range(len(input_vectors)):
                    data_point = input_vectors[i]
                    target = targets[i]
                    error = np.square(self.make_prediction(data_point) - target)
                    cumulative_error += error
                all_errors.append(np.sum(cumulative_error))

        return all_errors